In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv(r'E:\data science\mercari\train.tsv', delimiter='\t')

In [3]:
data['item'] = data['name']

import re

data['item'] = data['item'].map(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

df1 =data[['category_name','item']]

unknown_cat = df1[df1['category_name'].isnull()][:500]
known_cat = df1[df1['category_name'].notnull()][:60000]

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(known_cat['item'])
X_train_counts.shape

(60000, 17927)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(60000, 17927)

In [6]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, known_cat['category_name'])

In [7]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf.fit(known_cat['item'], known_cat['category_name'])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [8]:
from sklearn import metrics
predicted = text_clf.predict(unknown_cat['item'])
##np.mean(predicted == unknown_cat['category_name'])

text_clf.score(known_cat['item'], known_cat['category_name'])

0.44438333333333335

In [13]:
##from sklearn.model_selection import GridSearchCV
##parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
##gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
##gs_clf = gs_clf.fit(known_cat['item'], known_cat['category_name'])

##gs_clf.best_score_
##gs_clf.best_params_

In [11]:
from sklearn.ensemble import RandomForestClassifier
clf_rfc = RandomForestClassifier().fit(X_train_tfidf, known_cat['category_name'])
text_clf_rfc = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', RandomForestClassifier())])
text_clf_rfc.fit(known_cat['item'], known_cat['category_name'])

from sklearn import metrics
predicted_ = text_clf_rfc.predict(unknown_cat['item'])
unknown_cat['category_name'] = predicted 
text_clf_rfc.score(known_cat['item'], known_cat['category_name'])

0.95151666666666668

In [12]:
unknown_cat.head(10)

,category_name,item
122,Women/Tops & Blouses/T-Shirts,Bundle
155,Women/Tops & Blouses/T-Shirts,Nora Roberts Books
258,"Electronics/Cell Phones & Accessories/Cases, C...",ACER Laptop
984,Women/Women's Handbags/Shoulder Bag,AUTHENTIC BRWN MICHAEL KORS MAKEUP STAIN
1185,"Women/Athletic Apparel/Pants, Tights, Leggings",Teenage Mutant Ninja Turtle Van Extras
1274,Women/Tops & Blouses/Blouse,Black Ribbed Off the Shoulder Crop Top
1320,"Women/Athletic Apparel/Pants, Tights, Leggings",Nice Condition Pet Escort
1419,"Women/Athletic Apparel/Pants, Tights, Leggings",Toy Bundle
1491,Home/Home Décor/Home Décor Accents,Baby Sign
1521,Women/Tops & Blouses/T-Shirts,Nolan Ryan Patch Baseball Card
